# Subject Dependent
# Segment First

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import StratifiedKFold, ShuffleSplit
from tqdm.notebook import tqdm

from components.models import *
from components.helper import *
from components.dataset_jo import *
from components.train import *

import pickle
import numpy as np
import time
import argparse
import random
from scipy.stats import mode

## Training Configurations

In [2]:
class Config():
    def __init__(self):
        '''
        LSTM
        Conv1D_LSTM
        Conv1D_LSTM_Attention
        Conv1D_LSTM_SelfAttention
        '''
        
        
        # set running mode : juypyter or py
        # - jupyter = testing mode
        # - py      = production mode
        parser  = argparse.ArgumentParser()
        parser.add_argument('-a', '--model_name',     help='model_name' , type=str, required=False)
        parser.add_argument('-x', '--stim',           help='stim'       , type=int, required=False)
        parser.add_argument('-s', '--segment_number', help='segment_number'    , type=int, required=False)
        parser.add_argument('-l', '--len_reduction',  help='len_reduction' , type=str, required=False)
        parser.add_argument('-f', '--isdebug',        help='Set running mode' , type=str, required=False)
        args     = parser.parse_args()

        if args.isdebug == 'yes' or 'json' in args.isdebug :
            print("Jupyter mode")
            model_name     = 'LSTM'
            stim           = 1
            len_reduction  = 'mean'  # 'mean'  or 'sum' or 'last'
            segment_number = 1 # 1, 3, 5, 60

        else:
            model_name     = str(args.model_name)
            stim           = int(args.stim)
            segment_number = int(args.segment_number)
            len_reduction  = str(args.len_reduction)  # 'none' or 'mean' or 'sum' or 'last'
            

        
        
               
        
        ##============================================
        #  !!!!!!!!!!!!     DO NOT EDIT BELOW
        #============================================
        
        
        self.device = get_freer_gpu()
        # self.device = 'cpu'
        

        #========== Training Configurations==========
        self.path = "./data" 
        
        
        # STIMULI_VALENCE = 0
        # STIMULI_AROUSAL = 1       
        self.stim      = stim
        self.stim_name = 'AROUSAL' if self.stim else 'VALENCE'
        self.segment_number   = segment_number

        self.params     = {"batch_size" : 16, "shuffle" : True, "pin_memory" : True}
        self.num_epochs = 30
        self.lr         = 0.0001

        # true only if using 'LSTM'
        if model_name == 'LSTM' :
            self.seq_len_first = True
        else :
            self.seq_len_first = False

        self.debug = False
        if self.debug:
            self.num_epochs = 1
            self.n_split    = 3

        #========== Model Configurations==========
        # model list 

        
        
        self.model_name    = model_name   # this should be match with the model class
        self.input_dim     = 32   # we got 32 EEG channels
        self.hidden_dim    = 256  # let's define hidden dim as 256
        self.num_layers    = 2    # we gonna have two LSTM layers
        self.output_dim    = 1    # we got 2 classes so we can output only 1 number, 0 for first class and 1 for another class
        self.bidirectional = True # uses bidirectional LSTM
        self.dropout       = 0.5  # setting dropout to 0.5

        # for self attention
        self.len_reduction = len_reduction

        # for multi head attention
        self.n_heads       = 8
        self.d_k           = (self.hidden_dim * 2) // self.n_heads # (256 * 2) // 8
        
        if self.model_name == 'CNN2D' :
            if self.segment_number == 1:
                self.fc_shape = 237568
            if self.segment_number == 3:
                self.fc_shape = 73728
            if self.segment_number == 5:
                self.fc_shape = 40960
            if self.segment_number == 60:
                self.fc_shape = 2048
        
        
        #========== save config ==========
        self.segsplit      = 'dependent-seg'
        self.output_path   = f'./output/{self.segsplit}_{int(60/self.segment_number)}s/'
        if args.isdebug == 'yes' or 'json' in args.isdebug :
            self.result_csv    = f'{self.output_path}tmp_{self.model_name}_result.csv'
        else:
            self.result_csv    = f'{self.output_path}{self.model_name}_result.csv'
        

In [3]:
config = Config()
print_cls_var( config )

Jupyter mode
device : cuda:3
path : ./data
stim : 1
stim_name : AROUSAL
segment_number : 1
params : {'batch_size': 16, 'shuffle': True, 'pin_memory': True}
num_epochs : 20
lr : 0.0001
seq_len_first : True
debug : False
model_name : LSTM
input_dim : 32
hidden_dim : 256
num_layers : 2
output_dim : 1
bidirectional : True
dropout : 0.5
len_reduction : mean
n_heads : 8
d_k : 64
segsplit : dependent-seg
output_path : ./output/dependent-seg_60s/
result_csv : ./output/dependent-seg_60s/tmp_LSTM_result.csv


## Model Configurations

In [4]:
def init_model( config ):
    
    if config.model_name == 'LSTM' :
        model = LSTM( config.input_dim, 
                     config.hidden_dim, 
                     config.num_layers, 
                     config.output_dim, 
                     config.bidirectional, 
                     config.dropout)
        
    elif config.model_name == 'Conv1D_LSTM' :
        model = Conv1D_LSTM( config.input_dim, 
                            config.hidden_dim, 
                            config.num_layers, 
                            config.output_dim, 
                            config.bidirectional, 
                            config.dropout
                           )
    elif config.model_name == 'Conv1D_LSTM_Attention' :
        model = Conv1D_LSTM_Attention ( config.input_dim, 
                                       config.hidden_dim, 
                                       config.num_layers, 
                                       config.output_dim, 
                                       config.bidirectional, 
                                       config.dropout
                                      )

    elif config.model_name == 'Conv1D_LSTM_SelfAttention' :
        model = Conv1D_LSTM_SelfAttention( config.input_dim, 
                                  config.hidden_dim, 
                                  config.num_layers, 
                                  config.output_dim, 
                                  config.bidirectional, 
                                  config.dropout, 
                                  config.len_reduction   
                                 )
    elif config.model_name == 'Conv1D_LSTM_MultiHeadSelfAttention' :
        model =Conv1D_LSTM_MultiHeadSelfAttention( config.input_dim, 
                                                  config.hidden_dim, 
                                                  config.num_layers, 
                                                  config.output_dim, 
                                                  config.bidirectional, 
                                                  config.dropout, 
                                                  config.len_reduction,
                                                  config.n_heads,
                                                  config.d_k
                                                 )
    elif config.model_name == 'CNN2D' :
        model = CNN2D( config.input_dim, 
                       config.output_dim,
                      config.fc_shape 
                      
                    )
    
    
    model = model.to(config.device)  
    model.apply(initialize_weights)
    optimizer = optim.Adam(model.parameters(), lr=config.lr) 
    criterion = nn.BCEWithLogitsLoss()
    
    
    return model, optimizer, criterion

In [5]:
model, _, _ = init_model( config )
print(f'The model {type(model).__name__} has {count_parameters(model):,} trainable parameters')# Train the model

The model LSTM has 2,171,393 trainable parameters


In [6]:
dataset = Dataset_subjectDependent(config.path)
dataset.set_segment(config.segment_number)

filenames = dataset.get_file_list()
filenames.sort()
# print(filenames)

Found: 32 files


In [7]:
# def make_dataloader(X_orig, y_orig, trainval_idxs, test_idxs, params):
    
#     train_num = int(len(trainval_idxs)*0.75)
#     # print(train_num, len(trainval_idxs))
    
#     random.shuffle(trainval_idxs)
#     train_idxs = trainval_idxs[:train_num]
#     val_idxs   = trainval_idxs[train_num:]
    
#     assert [i for i in train_idxs if i in val_idxs] == []
    
#     X_train, X_val, X_test = X_orig[train_idxs], X_orig[val_idxs], X_orig[test_idxs]
#     y_train, y_val, y_test = y_orig[train_idxs], y_orig[val_idxs], y_orig[test_idxs]

#     train_dataset = TensorDataset(torch.tensor(X_train).float() , torch.tensor(y_train).float())
#     val_dataset   = TensorDataset(torch.tensor(X_val).float()   , torch.tensor(y_val).float())
#     test_dataset  = TensorDataset(torch.tensor(X_test).float()  , torch.tensor(y_test).float())
#     del X_train, X_val, X_test, y_train, y_val, y_test
    
#     print("len(train_dataset)", len(train_dataset))
#     print("len(val_dataset)  ", len(val_dataset))
#     print("len(test_dataset) ", len(test_dataset))

#     train_loader = DataLoader(train_dataset, **params)
#     val_loader   = DataLoader(val_dataset  , **params)
#     test_loader  = DataLoader(test_dataset , **params)
    
#     # print("len(train_loader)", len(train_loader))
#     # print("len(val_loader)  ", len(val_loader))
#     # print("len(test_loader) ", len(test_loader))
    
#     return train_loader, val_loader, test_loader

In [8]:
all_test_acc = []

for filename in filenames:
    # print("==================================================")
    # print("Participant : ", filename )
    
    # get each participant dataset without the groups
    if config.model_name == 'CNN2D':
        print("Getting spectrogram data ... ")
        X, y, _ = dataset.get_spec_data(filename, config.stim, return_type='numpy')
    else : 
        X, y, _ = dataset.get_data(filename, config.stim, return_type='numpy')
    # print(filename, X.shape, y.squeeze().shape)

    # ----------- OUTER CROSS VALIDATION -----------
    cv_outer = StratifiedKFold( n_splits = 10 )
    X_orig, y_orig = X.copy(), y.copy()

    for outer_fold, ( trainval_idxs, test_idxs ) in enumerate( cv_outer.split(X, y.squeeze())):

        # print( "Outer Fold : ", outer_fold )
        # print( trainval_idxs.shape, test_idxs.shape )

        X_trainval, X_test = X_orig[trainval_idxs], X_orig[test_idxs]
        y_trainval, y_test = y_orig[trainval_idxs], y_orig[test_idxs]
        
        test_dataset  = TensorDataset(torch.tensor(X_test).float()  , torch.tensor(y_test).float())
        test_loader   = DataLoader(test_dataset , **config.params)
        
        # ------------ INNER CROSS VALIDATION -------------
        cv_inner = ShuffleSplit( n_splits = 1 , train_size = 0.8, random_state = 42 )
        all_test_preds = []
        
        for inner_fold, ( train_idxs, val_idxs ) in enumerate( cv_inner.split(X_trainval, y_trainval.squeeze())):

            # print( "Inner Fold : ", inner_fold )
            # print( train_idxs.shape, val_idxs.shape )

            X_train, X_val = X_trainval[train_idxs], X_trainval[val_idxs]
            y_train, y_val = y_trainval[train_idxs], y_trainval[val_idxs]

            train_dataset = TensorDataset(torch.tensor(X_train).float() , torch.tensor(y_train).float())
            val_dataset   = TensorDataset(torch.tensor(X_val).float()   , torch.tensor(y_val).float())
            train_loader  = DataLoader(val_dataset  , **config.params)
            val_loader    = DataLoader(test_dataset , **config.params)

            # === Init MODEL ===
            model, optimizer, criterion = init_model( config )

            # === DO TRAINING === 
            train_loss, valid_loss, train_acc , valid_acc , test_loss, test_acc, best_epoch, epoch_times = train(config.num_epochs,
                                                                                                     model,
                                                                                                     train_loader,
                                                                                                     val_loader,
                                                                                                     test_loader,
                                                                                                     optimizer,
                                                                                                     criterion,
                                                                                                     config.device,
                                                                                                     config.seq_len_first)

            all_test_acc.append(test_acc)
            del model, optimizer, criterion, train_loader, val_loader

        # save to csv at specific epoch
            for epoch in range( best_epoch + 1 ) :
                result_csv_dic               = {}
                result_csv_dic['segment_number'] =  config.segment_number
                result_csv_dic['len_reduction']  =  config.len_reduction
                result_csv_dic['par']         =  filename
                result_csv_dic['stim_name']   =  config.stim_name
                result_csv_dic['fold']        =  outer_fold
                result_csv_dic['epoch']       =  epoch
                result_csv_dic['train_loss']  = train_loss[epoch]
                result_csv_dic['valid_loss']  = valid_loss[epoch]
                result_csv_dic['train_acc']   = train_acc[epoch]
                result_csv_dic['valid_acc']   = valid_acc[epoch]
                result_csv_dic['epoch_times'] =  epoch_times[epoch]

                if epoch == best_epoch:
                    result_csv_dic['test_loss'] = test_loss
                    result_csv_dic['test_acc']  = test_acc
                else:
                    result_csv_dic['test_loss'] = ''
                    result_csv_dic['test_acc']  = ''

                save_result_csv( result_csv_dic, config.result_csv )

/opt/conda/lib/python3.9/site-packages/torch/nn/modules/rnn.py:691: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at  ../aten/src/ATen/native/cudnn/RNN.cpp:925.)
  result = _VF.lstm(input, hx, self._flat_weights, self.bias, self.num_layers,


FINAL Best Model from Best Epoch 19 Test Loss = 0.53005450963974, Test Acc = 0.75
FINAL Best Model from Best Epoch 19 Test Loss = 0.5710050463676453, Test Acc = 0.75


KeyboardInterrupt: 

In [ ]:
print("="*50)
print("="*50)

print("segsplit      ", config.segsplit)
print("model_name    ", config.model_name)
print("stim_name     ", config.stim_name)
print("segment size  ", int(60/config.segment_number), 's')

print("all_test_acc : ", all_test_acc)
print("AVG all_test_acc : ", np.mean(all_test_acc))
print("SD  all_test_acc : ", np.std(all_test_acc))

print("="*50)
print("="*50)

In [ ]:
# seg_num = 1 : (40, 32, 119, 65)
# seg_num = 3 : (120, 32, 39, 65)
# seg_num = 5 : (200, 32, 23, 65)
# seg_num = 60 : (2400, 32, 31, 5)